In [2]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [3]:
df= pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [4]:
df.head(2)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0


In [5]:
launch_site_names = df['Launch Site'].unique()

# Create a list of dict-like option objects for dropdown
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_site_names]


app = dash.Dash(__name__)

app.layout = html.Div(children=[ html.H1("SpaceX lanch records Dashboard",
                                style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
                                 dcc.Dropdown(id='site-dropdown',  # id attribute
                                     options=dropdown_options,  # options attribute
                                     value='ALL',  # default dropdown value
                                     placeholder="Select a Launch Site here",  # placeholder attribute
                                     searchable=True  # searchable attribute
                                 ),dcc.Graph(id='success-pie-chart'),
                                  dcc.RangeSlider(
                                    id='payload-slider',  # id attribute
                                    min=0,  # starting point
                                    max=10000,  # ending point
                                    step=1000,  # interval on the slider
                                    marks={0: '0', 10000: '10000'},  # optional marks on the slider
                                    value=[df["Payload Mass (kg)"].min(),df["Payload Mass (kg)"].max()]  # initial selected range
                                      
                                      ),dcc.Graph(id='success-scatter plot'),
                               ])
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # All sites selected, show the total success launches
        fig = px.pie(df, values='class', names='Launch Site', title='Total Success Launches for All Sites')
    else:
        # Specific site selected, filter the DataFrame for that site and show success/fail counts
        filtered_df = df[df["Launch Site"] == selected_site]
        fig = px.pie(filtered_df, names='class', title=f'Success/Failure Launches for {selected_site}')

    return fig
@app.callback(
    Output(component_id='success-scatter plot', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(selected_site, selected_payload_range):
    # Filter the DataFrame based on selected launch site
    if selected_site == 'ALL':
        filtered_df = df
    else:
        filtered_df = df[df['Launch Site'] == selected_site]

    # Filter the DataFrame based on selected payload range
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= selected_payload_range[0]) &
                              (filtered_df['Payload Mass (kg)'] <= selected_payload_range[1])]

    # Create scatter plot
    fig = px.scatter(filtered_df,
                     x='Payload Mass (kg)',
                     y='class',
                     color='Booster Version Category',
                     title=f'Scatter Plot for Payload and Launch Outcome ({selected_site})',
                     labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Launch Outcome'},
                     )

    return fig
if __name__ == '__main__':
    app.run_server()            